In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [3]:
df=pd.read_csv('Ridership and Features.csv')
df.head()

,Unnamed: 0,BCT_Code,MonthS,DOWS,Ridership,Borough,Total population,Unemployment_Rate,Per_Capita_Income,18-64_Poverty
0,0,1000201,1,0,228,Manhattan,2012.0,5.1%,"13,405",29.1%
1,1,1000201,1,1,195,Manhattan,2012.0,5.1%,"13,405",29.1%
2,2,1000201,1,2,230,Manhattan,2012.0,5.1%,"13,405",29.1%
3,3,1000201,1,3,229,Manhattan,2012.0,5.1%,"13,405",29.1%
4,4,1000201,1,4,191,Manhattan,2012.0,5.1%,"13,405",29.1%


In [4]:
df.shape

(59277, 10)

In [5]:
df['Unemployment_Rate'] = df['Unemployment_Rate'].str.replace('%','')
df['18-64_Poverty'] = df['18-64_Poverty'].str.replace('%','')
df['Per_Capita_Income'] = df['Per_Capita_Income'].str.replace(',','')
df['Unemployment_Rate'] = pd.to_numeric(df['Unemployment_Rate'], errors='coerce')
df['18-64_Poverty'] = pd.to_numeric(df['18-64_Poverty'], errors='coerce')
df['Per_Capita_Income'] = pd.to_numeric(df['Per_Capita_Income'], errors='coerce')

In [6]:
df=df.dropna()

In [7]:
df.columns

Index(['Unnamed: 0', 'BCT_Code', 'MonthS', 'DOWS', 'Ridership', 'Borough',
       'Total population', 'Unemployment_Rate', 'Per_Capita_Income',
       '18-64_Poverty'],
      dtype='object')

In [8]:
col = df.columns.tolist()
col = [col[1]] + [col[5]] + [col[4]] + col[2:4] + col[6:]
df=df[col]
df.head(3)

,BCT_Code,Borough,Ridership,MonthS,DOWS,Total population,Unemployment_Rate,Per_Capita_Income,18-64_Poverty
0,1000201,Manhattan,228,1,0,2012.0,5.1,13405.0,29.1
1,1000201,Manhattan,195,1,1,2012.0,5.1,13405.0,29.1
2,1000201,Manhattan,230,1,2,2012.0,5.1,13405.0,29.1


In [9]:
df.to_csv('Ridership with Features.csv')
## 这个文件重新上传了

In [10]:
df=df.reset_index()
df=df.iloc[:,1:]

In [11]:
month=pd.get_dummies(df['MonthS'], dtype=float, prefix='month')
dow=pd.get_dummies(df['DOWS'], dtype=float, prefix='dow')

In [12]:
#sc = StandardScaler()
#df_sc = sc.fit_transform(df.iloc[:,2:])

X_sc = df.iloc[:,5:9].values
y = df.iloc[:,2].values
y = y.reshape(-1,1)

sc_X = StandardScaler()
sc_y = StandardScaler()

X_sc = sc_X.fit_transform(X_sc)
y = sc_y.fit_transform(y)

In [13]:
X = pd.DataFrame(X_sc,columns=df.columns[5:9])
X=X.join(month)
X=X.join(dow)
X=X.values

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
X_train1, X_val, y_train1, y_val = train_test_split(X, y, test_size=0.25, random_state=123)

#### Linear

In [ ]:
sr = svm.SVR(kernel = 'linear')
sr.fit(X_train, y_train.ravel())

SVR(kernel='linear')

In [ ]:
y_pred = sr.predict(X_test)
y_pred1 = sr.predict(X_train)

y_pred11 = y_pred.reshape(-1,1)
y_pred111 = y_pred1.reshape(-1,1)

y_pred11 = sc_y.inverse_transform(y_pred11)
y_pred111 = sc_y.inverse_transform(y_pred111)
print(r2_score(y_test, y_pred))
print(r2_score(y_train, y_pred1))

0.2630184932702594
0.2555763531044739


#### Poly

In [ ]:
sr = svm.SVR(kernel = 'poly')
sr.fit(X_train, y_train.ravel())

y_pred = sr.predict(X_test)
y_pred1 = sr.predict(X_train)

y_pred11 = y_pred.reshape(-1,1)
y_pred111 = y_pred1.reshape(-1,1)

y_pred11 = sc_y.inverse_transform(y_pred11)
y_pred111 = sc_y.inverse_transform(y_pred111)
print(r2_score(y_test, y_pred))
print(r2_score(y_train, y_pred1))

0.41115113903290845
0.4142623895316443


#### RBF

In [ ]:
sr = svm.SVR(kernel = 'rbf')
sr.fit(X_train, y_train.ravel())

y_pred = sr.predict(X_test)
y_pred1 = sr.predict(X_train)

y_pred11 = y_pred.reshape(-1,1)
y_pred111 = y_pred1.reshape(-1,1)

y_pred11 = sc_y.inverse_transform(y_pred11)
y_pred111 = sc_y.inverse_transform(y_pred111)
print(r2_score(y_test, y_pred))
print(r2_score(y_train, y_pred1))

0.4737311943494871
0.4854948046201206


#### Sigmoid

In [ ]:
sr = svm.SVR(kernel = 'sigmoid')
sr.fit(X_train, y_train.ravel())

y_pred = sr.predict(X_test)
y_pred1 = sr.predict(X_train)

y_pred11 = y_pred.reshape(-1,1)
y_pred111 = y_pred1.reshape(-1,1)

y_pred11 = sc_y.inverse_transform(y_pred11)
y_pred111 = sc_y.inverse_transform(y_pred111)
print(r2_score(y_test, y_pred))
print(r2_score(y_train, y_pred1))

-391157.75474676094
-407867.77213028684


### Choose rbf

In [1]:
param_grid ={'kernel':["rbf"],
            'C':10**np.linspace(-5,10,4),
            'gamma':np.linspace(-5,10,4),
            'coef0':10**np.linspace(-3,3,4),
            'degree':range(1,4)}

sr = svm.SVR()
gr=GridSearchCV(sr,param_grid=param_grid, refit=True)
rs=gr.fit(X_train1,y_train1.ravel())
print(rs.best_params_)

NameError: ignored

In [ ]:
10**np.linspace(-5,10,5)

array([1.00000000e-05, 5.62341325e-02, 3.16227766e+02, 1.77827941e+06,
       1.00000000e+10])

In [ ]:
np.linspace(-5,10,5)

In [ ]:
10**np.linspace(-3,3,5)

In [ ]:
y_pred = sr.predict(X_val)
y_pred1 = sr.predict(X_train1)

y_pred11 = y_pred.reshape(-1,1)
y_pred111 = y_pred1.reshape(-1,1)

y_pred11 = sc_y.inverse_transform(y_pred11)
y_pred111 = sc_y.inverse_transform(y_pred111)
print(r2_score(y_val, y_val))
print(r2_score(y_train1, y_pred1))